<a href="https://colab.research.google.com/github/AmirKhan-45/piaic-batch61/blob/main/LANGCHAIN_RAG_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00


In [7]:
from google.colab import userdata


from pinecone import Pinecone, ServerlessSpec


pinecone_api_key = userdata.get('PINECONE_API_KEY')


pc = Pinecone(api_key=pinecone_api_key)

In [8]:
import time

index_name = "langchain-rag-project"  # change if desired


pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


index = pc.Index(index_name)

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [10]:
vector = embeddings.embed_query("we are building a RAG Text")


In [12]:
vector[:4]

[0.04060795158147812,
 -0.04385491460561752,
 -0.05683692544698715,
 -0.021194666624069214]

In [13]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [14]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [15]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [16]:

# Data save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


# Data Retrive

In [17]:
from uuid import uuid4
uuid4()

UUID('ba24ea82-787f-4604-b8d0-ef316613f520')

In [18]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['c898fddd-78ea-4b73-9b6c-52f6b6fa3fa3',
 '75500a6c-32d9-49f4-a5a0-69022a85cdfe',
 '1e91611a-9957-4abb-a567-b4e70894fe8b',
 'a7d2bc05-d534-4668-9f9e-542b5c79c051',
 'f9a51439-eb0e-4d42-9845-d8931909f5d8',
 '6e2f47a4-a401-4d47-a7a8-5f7928a1cc59',
 '65d79d5e-cde2-4fae-9d91-c51f83a7d3b3',
 '9419d5d2-4480-4e5b-9eb5-f7509082d636',
 'd55b726e-731f-48c6-86fa-b30f99c37fff',
 'a890a288-0519-4da6-ab11-a12c4e6b4753']

In [19]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [20]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.537337] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.533720] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [22]:
def answer_to_user(query: str):

  vector_result = vector_store.similarity_search(query, k=2)

  final_answer = llm.invoke(f"Answer this user query {query}, Here are some reference to answer{vector_result}")

  return final_answer

In [24]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

In [25]:
answer.content

"The provided text mentions LangChain in the context of building projects.  One tweet expresses excitement about using LangChain for a new project.  The other tweet promotes a different framework, LangGraph, for building stateful, agentic applications.  Neither tweet directly addresses LangChain's abstractions for simplifying LLM work, although the second tweet implicitly suggests that LangChain is a framework for building applications that likely involve LLMs.\n"